# Real-time software architecture

Software architecture simply refers to structure of the software that is being designed.

To comply with the real-time requirements, the following software architectures are possible.

## Simple polled loop

This is simply a repeated testing of a flag to check whether an event occurred.


``` python
while True:
    if ok:
        task1()
```

| Ideal condition | Cons |
| :--- | :--- |
| CPU is dedicated to handling the I/O for some fast device | Fails when there is a burst of events |
| Overlapping of events don't occur | Unable to handle complex systems |
| | Waste CPU time on polling (especially with infrequent events) |

## Round robin

A set of $n$ self-contained tasks.
The all tasks are processed at the same frequency.

``` python
while True:
    if flag1:
        task1()
    if flag2:
        task2()
    if flag3:
        task3()
```

### Changing cycle rates

To modify the cycle rates, we can repeat the task multiple times in the same cycle.
This way, we have some task which are run more often than others.

``` python
while True:
    task1()
    task2()
    task3()
    task2()
```    

| Ideal condition| Cons |
| :--- | :--- |
| There are few devices| Fails when some device has a deadline that is shorter than the time needed to process the whole loop |
| Frequency of servicing each device is same | Rather fragile, adding another device can easily violate the deadline of some task |
| Tasks are short | |
| No tight time requirements | 

## Round robin with interrupts

ISR read the device status and sets a flag.
The main loop with the poll on these flags and perform relevant actions if the flag is set.

``` python
flag1, flag2 = False, False

while True:
    if flag1:
        flag1 = False
        task1()
    if flag2:
        flag2 = False
        task2()

def ISR1():
    flag1 = True
    
def ISR2():
    flag2 = True
```

This has improvements over round robin, because we immediately attend to the device that requires attention.
It also gives more flexibility.

However, it is possible that some high priority task (`task2`) becomes blocked because we are processing a lower priority task (`task1`).

## Function queue scheduling

This is similar to round robin with interrupt, but we use a pointer to a function instead of a flag.
Then, we push these pointers to a queue when a device is ready.

``` python
func_queue = [] # can be a priority queue to allow prioritizing of tasks

while True:
    for task in func_queue:
        task()
    func_queue = []

def ISR1():
    func_queue.push(task1)
    
def ISR2():
    func_queue.push(task2)
```    

It allows us to implement some priority system into the task, but still suffers the same issue as round robin with interrupts.

## Real-time operation system

RTOS provides an infrastructure to handle multiple tasks, using task scheduling and dispatching.

| Pros | Cons |
| :--- | :--- |
| Provides a clean and convenient way to control complex applications | Is relatively huge and must be customized |
| Audited to be reliable | |